In [7]:
%%writefile utility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re
import gzip
import numpy as np

################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting utility.py


In [8]:
%%writefile schemafile.yaml
file_type: csv
dataset_name: cabdatafile
file_name: Cab_Data
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - transaction_id
    - date_of_travel
    - company
    - city
    - km_travelled
    - price_charged
    - cost_of_trip

Overwriting schemafile.yaml


In [9]:
# Read config file
import utility as util
config_data = util.read_config_file("schemafile.yaml")

In [10]:
config_data['inbound_delimiter']

','

In [11]:
#inspecting data of config file
config_data

{'columns': ['transaction_id',
  'date_of_travel',
  'company',
  'city',
  'km_travelled',
  'price_charged',
  'cost_of_trip'],
 'dataset_name': 'cabdatafile',
 'file_name': 'Cab_Data',
 'file_type': 'csv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'table_name': 'edsurv'}

In [12]:
# Normal reading process of the file
import pandas as pd
df_sample = pd.read_csv("/Cab_Data.csv",delimiter=',')
df_sample.head()

,Transaction ID,Date of Travel,Company,City,KM Travelled,Price Charged,Cost of Trip
0,10000011,42377,Pink Cab,ATLANTA GA,30.45,370.95,313.635
1,10000012,42375,Pink Cab,ATLANTA GA,28.62,358.52,334.854
2,10000013,42371,Pink Cab,ATLANTA GA,9.04,125.20,97.632
3,10000014,42376,Pink Cab,ATLANTA GA,33.17,377.40,351.602
4,10000015,42372,Pink Cab,ATLANTA GA,8.73,114.62,97.776


In [13]:
# read the file using config file
file_type = config_data['file_type']
source_file = "/" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,Transaction ID,Date of Travel,Company,City,KM Travelled,Price Charged,Cost of Trip
0,10000011,42377,Pink Cab,ATLANTA GA,30.45,370.95,313.635
1,10000012,42375,Pink Cab,ATLANTA GA,28.62,358.52,334.854
2,10000013,42371,Pink Cab,ATLANTA GA,9.04,125.20,97.632
3,10000014,42376,Pink Cab,ATLANTA GA,33.17,377.40,351.602
4,10000015,42372,Pink Cab,ATLANTA GA,8.73,114.62,97.776


In [14]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation passed


1

In [15]:
count_row = df.shape[0]  # Gives number of rows
count_col = df.shape[1]  # Gives number of columns
import os
size = os.path.getsize('/Cab_Data.csv') # get the size of file

In [16]:
print("Total Number of Rows:",count_row)
print("Total Number of Columns:",count_col)
print("Size of the file:",size)

Total Number of Rows: 359392
Total Number of Columns: 7
Size of the file: 21158610


In [17]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['transaction_id', 'date_of_travel', 'company', 'city', 'km_travelled',
       'price_charged', 'cost_of_trip'],
      dtype='object')
columns of YAML are: ['transaction_id', 'date_of_travel', 'company', 'city', 'km_travelled', 'price_charged', 'cost_of_trip']


In [18]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")
    df.to_csv(r'cab.txt', header=None, index=None, sep='|', mode='a')
    f_in = open('cab.txt')
    import gzip
    import shutil
    with open('cab.txt', 'rt') as f_in:
        with gzip.open('cab.txt.gz', 'wt') as f_out:
            shutil.copyfileobj(f_in, f_out) 
    

column name and column length validation passed
col validation passed
